<a href="https://colab.research.google.com/github/mrkidding/Magic-trick/blob/main/%EF%BC%888%2C_5%2C_2)_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hide 3 cards, compute the upper bound
Accoring to paper 1, N - m + 3 choose 3 <= (m - 3)! * (m choose 3).
For given m, let's compute the largest N

In [ ]:
import math

def compute_N(m):
  factorial = math.factorial(m)
  i = m
  while (i - m + 3) * (i - m + 2) * (i - m + 1) <= factorial:
    i = i + 1
  return i - 1

In [ ]:
for i in range(4, 9):
  print("m:", i, ", N: ", compute_N(i))

m: 4 , N:  5
m: 5 , N:  8
m: 6 , N:  13
m: 7 , N:  22
m: 8 , N:  40


# Let's talk about the case that m = 5 first


In [ ]:
#U: Display of k(2) cards, (8 choose 2) * 2 = 56.
#V: a set of m(5) cards, (8 choose 5) = 56.
#|U| >= |V|.


from itertools import combinations
from itertools import permutations

n = 7
cards = [_ for _ in range(1, n + 1)]

# 获取所有有序取出2张牌的排列
pers = list(permutations(cards, 2))
print(len(pers))

per_set = set(pers)

# 获取所有有序取出5张牌的组合
combs = list(combinations(cards, 5))
print(len(combs))

comb_set = set(combs)
# 输出结果
#for combination in combinations:
#    print(combination)

42
21


# Use HOO method to get a comb from a permutation


In [ ]:
import copy
import bisect

def recover_2_to_5(per):
  index = 1 if per[1] < per[0] else 0
  temp_list = copy.deepcopy(list(per))
  temp_list.sort()

  # 2 => 3
  potential_3 = []
  for i in range(1, n + 1):
    if i in per:
      continue
    sum_list = sum(per) + i
    reminder = sum_list % 3
    location = bisect.bisect_left(temp_list, i)
    if reminder == location:
      potential_3.append(i)

  if index == 1 and len(potential_3) == 1:
    return

  temp_list.append(potential_3[index])
  temp_list.sort()
  #print("2 => 3: ", temp_list)

  # 3 => 4
  for i in range(1, n + 1):
    if i in temp_list:
      continue
    sum_list = sum(temp_list) + i
    reminder = sum_list % 4
    location = bisect.bisect_left(temp_list, i)
    if reminder == location:
      temp_list.append(i)
      break
  temp_list.sort()
  #print("3 => 4: ", temp_list)

  # 4 => 5
  for i in range(1, n + 1):
    if i in temp_list:
      continue
    sum_list = sum(temp_list) + i
    reminder = sum_list % 5
    location = bisect.bisect_left(temp_list, i)
    if reminder == location:
      temp_list.append(i)
      break

  temp_list.sort()
  #print("4 => 5: ", temp_list)

  return temp_list



In [ ]:
copy_per_set = copy.deepcopy(per_set)
copy_comb_set = copy.deepcopy(comb_set)
for per in pers:
  temp_list = recover_2_to_5(per)
  if temp_list is None or len(temp_list) != 5:
    continue
  per_5 = tuple(temp_list)
  copy_per_set.remove(per)
  copy_comb_set.remove(per_5)

# the left unmatched permutations and combinations
print(copy_per_set)
print(len(copy_per_set))
print(copy_comb_set)
print(len(copy_comb_set))

{(3, 4), (4, 3), (5, 4), (5, 1), (5, 7), (1, 3), (6, 5), (4, 2), (5, 3), (1, 2), (2, 1), (6, 4), (7, 3), (6, 7), (7, 6), (3, 5), (1, 4), (2, 3), (7, 2), (7, 5), (6, 3)}
21
set()
0


# Match the left permutations and combinations

In [ ]:
# we have 12 left permutations and combinations, use grubi to match them
# define yij = 1 if comb j contains per i
# max Sum(xij)
# s.t. Sum over i (xij) = 1, for any j
#      Sum over j (xij) = 1, for any i
#      xij <= yij

%pip install gurobipy

import gurobipy as gp
from gurobipy import GRB



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 66.5 MB/s eta 0:00:00


In [ ]:
m = gp.Model("left_matching_8_5_2")

# Create decision variables using a list of indices from 1 to 144
indices = list(range(0, 144))

x = m.addVars(indices, vtype=GRB.BINARY, name='x')

# Objective function
m.setObjective(- x.sum())


Restricted license - for non-production use only - expires 2025-11-24


In [ ]:
# construct yij
per_list = list(copy_per_set)
comb_list = list(copy_comb_set)

y = [0 for _ in range(144)]

for i in range(12):
  for j in range(12):
    is_subset = set(per_list[i]).issubset(set(comb_list[j]))
    if is_subset:
      y[i * 12 + j] = 1


IndexError: list index out of range

In [ ]:
# constraints
for i in range(12):
  m.addConstr(gp.quicksum(x[12 * i + j] for j in range(12)) <= 1, 'x_j_sum_{}'.format(j))

for j in range(12):
  m.addConstr(gp.quicksum(x[12 * i + j] for i in range(12)) <= 1, 'x_i_sum_{}'.format(i))

for i in range(144):
  m.addConstr(x[i] <= y[i])



In [ ]:
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 168 rows, 144 columns and 432 nonzeros
Model fingerprint: 0x4321beb7
Variable types: 0 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.0000000
Presolve removed 168 rows and 144 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [ ]:
for i in range(12):
  for j in range(12):
    if x[i * 12 + j].X == 1:
      print("per: ", per_list[i], ", comb: ", comb_list[j])

per:  (3, 4) , comb:  (3, 4, 5, 7, 8)
per:  (5, 4) , comb:  (2, 4, 5, 7, 8)
per:  (8, 6) , comb:  (2, 3, 6, 7, 8)
per:  (1, 3) , comb:  (1, 2, 3, 7, 8)
per:  (8, 5) , comb:  (2, 4, 5, 6, 8)
per:  (1, 2) , comb:  (1, 2, 3, 6, 8)
per:  (6, 4) , comb:  (1, 2, 4, 6, 8)
per:  (7, 6) , comb:  (4, 5, 6, 7, 8)
per:  (3, 5) , comb:  (2, 3, 5, 7, 8)
per:  (8, 4) , comb:  (1, 4, 6, 7, 8)
per:  (1, 4) , comb:  (1, 2, 4, 5, 8)
per:  (7, 8) , comb:  (1, 2, 4, 7, 8)


So now, remember the 12 matchings and use the method HOO, we can get the upper bound of 8, 5, 2 model